In [2]:
from cgmbrush import *
from tempfile import TemporaryFile
import cProfile
import io
import pstats

In [ ]:
# Perf testing for reading Bolshoi density fields

provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
den_field = provider.get_density_field(0, 256)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

saveArray("perf_denfield", den_field)

In [ ]:
provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
read_denfield = loadArray("perf_denfield.npy")
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf_read.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

In [ ]:
# Testing savez
outfile = TemporaryFile()
x = np.arange(1000)
y = np.arange(1000) + 1

arrays = {'y':y, 'x':x, }

np.savez(outfile, **arrays)
_ = outfile.seek(0)
npzfile = np.load(outfile)
npzfile['x']

In [3]:
p = BolshoiProvider()
a = p.extract_halos(0)
print(a.shape)
b = loadArray('bol_halos_0.npy')
print(b.shape)

(8949374, 10)
(8949374, 10)


In [10]:
print(a.shape)
print(a.info(memory_usage='deep'))
print(b.shape)
print(type(b))
print(str((b.size * b.itemsize)/1024**2) + " MB")
print(b[0])

(8949374, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8949374 entries, 0 to 8949373
Data columns (total 10 columns):
row_id    int64
x         float64
y         float64
z         float64
Mvir      int64
Mtot      int64
Rvir      float64
ix        int64
iy        int64
iz        int64
dtypes: float64(4), int64(6)
memory usage: 682.8 MB
None
(8949374, 10)
<class 'numpy.ndarray'>
682.7830505371094 MB
[1.00000000e+00 3.72420001e+00 2.06110001e+00 2.85319996e+00
 3.11600000e+09 3.25200000e+09 2.41999999e-02 1.50000000e+01
 8.00000000e+00 1.10000000e+01]


In [ ]:
# Profiling full convolution code for simplest case

min_mass = 10**10 # TODO into config
max_mass = 10**14.5
log_bins = 30
config = Configuration('tophat_spherical', 1, resolution=1, file_prefix='STH', load_from_files=False)
config.provider = BolshoiProvider()
filename = config.file_prefix + str(config.resolution) + '_' + str(config.den_grid_size)


pr = cProfile.Profile()
pr.enable()
config.results = hist_profile(config.provider, config.den_grid_size, config.RS_array, min_mass, 
                                    max_mass, log_bins, config.subtraction_halo_profile, 
                                    config.addition_halo_profile, config.scaling_radius, config.resolution)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'perf_convo_' + filename + '_v1.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())
     


